# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f05005a2670>
├── 'a' --> tensor([[ 0.2928, -0.5008, -0.0026],
│                   [-0.2707, -1.5274,  0.0615]])
└── 'x' --> <FastTreeValue 0x7f05005a2eb0>
    └── 'c' --> tensor([[ 0.3028, -0.1026, -1.3381,  1.0158],
                        [-1.2644,  0.1674, -1.1713, -0.5053],
                        [-1.0877, -0.3578, -1.7255,  0.1743]])

In [4]:
t.a

tensor([[ 0.2928, -0.5008, -0.0026],
        [-0.2707, -1.5274,  0.0615]])

In [5]:
%timeit t.a

59.8 ns ± 0.0571 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f05005a2670>
├── 'a' --> tensor([[-1.2533, -0.2815,  1.7095],
│                   [-0.9207,  1.0024,  1.3512]])
└── 'x' --> <FastTreeValue 0x7f05005a2eb0>
    └── 'c' --> tensor([[ 0.3028, -0.1026, -1.3381,  1.0158],
                        [-1.2644,  0.1674, -1.1713, -0.5053],
                        [-1.0877, -0.3578, -1.7255,  0.1743]])

In [7]:
%timeit t.a = new_value

58.2 ns ± 0.0336 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2928, -0.5008, -0.0026],
               [-0.2707, -1.5274,  0.0615]]),
    x: Batch(
           c: tensor([[ 0.3028, -0.1026, -1.3381,  1.0158],
                      [-1.2644,  0.1674, -1.1713, -0.5053],
                      [-1.0877, -0.3578, -1.7255,  0.1743]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2928, -0.5008, -0.0026],
        [-0.2707, -1.5274,  0.0615]])

In [11]:
%timeit b.a

56.5 ns ± 0.0289 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0511, -0.0190,  0.1257],
               [ 1.3128, -1.3863,  1.2278]]),
    x: Batch(
           c: tensor([[ 0.3028, -0.1026, -1.3381,  1.0158],
                      [-1.2644,  0.1674, -1.1713, -0.5053],
                      [-1.0877, -0.3578, -1.7255,  0.1743]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.106 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

844 ns ± 0.0687 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.8 µs ± 38.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 542 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0448e0fb80>
├── 'a' --> tensor([[[ 0.2928, -0.5008, -0.0026],
│                    [-0.2707, -1.5274,  0.0615]],
│           
│                   [[ 0.2928, -0.5008, -0.0026],
│                    [-0.2707, -1.5274,  0.0615]],
│           
│                   [[ 0.2928, -0.5008, -0.0026],
│                    [-0.2707, -1.5274,  0.0615]],
│           
│                   [[ 0.2928, -0.5008, -0.0026],
│                    [-0.2707, -1.5274,  0.0615]],
│           
│                   [[ 0.2928, -0.5008, -0.0026],
│                    [-0.2707, -1.5274,  0.0615]],
│           
│                   [[ 0.2928, -0.5008, -0.0026],
│                    [-0.2707, -1.5274,  0.0615]],
│           
│                   [[ 0.2928, -0.5008, -0.0026],
│                    [-0.2707, -1.5274,  0.0615]],
│           
│                   [[ 0.2928, -0.5008, -0.0026],
│                    [-0.2707, -1.5274,  0.0615]]])
└── 'x' --> <FastTreeValue 0x7f050059b850>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f044883bf40>
├── 'a' --> tensor([[ 0.2928, -0.5008, -0.0026],
│                   [-0.2707, -1.5274,  0.0615],
│                   [ 0.2928, -0.5008, -0.0026],
│                   [-0.2707, -1.5274,  0.0615],
│                   [ 0.2928, -0.5008, -0.0026],
│                   [-0.2707, -1.5274,  0.0615],
│                   [ 0.2928, -0.5008, -0.0026],
│                   [-0.2707, -1.5274,  0.0615],
│                   [ 0.2928, -0.5008, -0.0026],
│                   [-0.2707, -1.5274,  0.0615],
│                   [ 0.2928, -0.5008, -0.0026],
│                   [-0.2707, -1.5274,  0.0615],
│                   [ 0.2928, -0.5008, -0.0026],
│                   [-0.2707, -1.5274,  0.0615],
│                   [ 0.2928, -0.5008, -0.0026],
│                   [-0.2707, -1.5274,  0.0615]])
└── 'x' --> <FastTreeValue 0x7f044880acd0>
    └── 'c' --> tensor([[ 0.3028, -0.1026, -1.3381,  1.0158],
                        [-1.2644,  0.1674, -1.1713, -0.5053],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 85.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.8 µs ± 57.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2928, -0.5008, -0.0026],
                [-0.2707, -1.5274,  0.0615]],
       
               [[ 0.2928, -0.5008, -0.0026],
                [-0.2707, -1.5274,  0.0615]],
       
               [[ 0.2928, -0.5008, -0.0026],
                [-0.2707, -1.5274,  0.0615]],
       
               [[ 0.2928, -0.5008, -0.0026],
                [-0.2707, -1.5274,  0.0615]],
       
               [[ 0.2928, -0.5008, -0.0026],
                [-0.2707, -1.5274,  0.0615]],
       
               [[ 0.2928, -0.5008, -0.0026],
                [-0.2707, -1.5274,  0.0615]],
       
               [[ 0.2928, -0.5008, -0.0026],
                [-0.2707, -1.5274,  0.0615]],
       
               [[ 0.2928, -0.5008, -0.0026],
                [-0.2707, -1.5274,  0.0615]]]),
    x: Batch(
           c: tensor([[[ 0.3028, -0.1026, -1.3381,  1.0158],
                       [-1.2644,  0.1674, -1.1713, -0.5053],
                       [-1.0877, -0.3578, -1.7255,  0.1743]],
         

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 151 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2928, -0.5008, -0.0026],
               [-0.2707, -1.5274,  0.0615],
               [ 0.2928, -0.5008, -0.0026],
               [-0.2707, -1.5274,  0.0615],
               [ 0.2928, -0.5008, -0.0026],
               [-0.2707, -1.5274,  0.0615],
               [ 0.2928, -0.5008, -0.0026],
               [-0.2707, -1.5274,  0.0615],
               [ 0.2928, -0.5008, -0.0026],
               [-0.2707, -1.5274,  0.0615],
               [ 0.2928, -0.5008, -0.0026],
               [-0.2707, -1.5274,  0.0615],
               [ 0.2928, -0.5008, -0.0026],
               [-0.2707, -1.5274,  0.0615],
               [ 0.2928, -0.5008, -0.0026],
               [-0.2707, -1.5274,  0.0615]]),
    x: Batch(
           c: tensor([[ 0.3028, -0.1026, -1.3381,  1.0158],
                      [-1.2644,  0.1674, -1.1713, -0.5053],
                      [-1.0877, -0.3578, -1.7255,  0.1743],
                      [ 0.3028, -0.1026, -1.3381,  1.0158],
                      [-1.2644,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 462 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

317 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
